# Notebook Overview — Ensuring Complete Cross-Sectional Coverage Before the Pipeline

This notebook prepares the raw CRSP equity data from the CSV File to a Pickle File for the later preprocessing and labeling pipeline by enforcing strict cross-sectional consistency across the trading universe. Before any feature construction, normalization, or event-based labeling is applied, the notebook filters the dataset to retain only those stocks that have valid observations on every trading day within the selected sample period. This guarantees that all downstream feature matrices, returns, volatility estimates, and eventual labels are aligned across a uniform time index, preventing gaps and misaligned cross-sectional batches.

The Resulting Dataframe contains 2726479 Entries over multiple US Stocks. Each Stock has 2768 Entries. 

In [1]:
import pickle as pkl
import pandas as pd


In [2]:
data = pd.read_csv(".././Data/crsp_us.csv")

In [3]:
data = data.rename(columns={"DlyCalDt": "Date"})
data["Date"] = pd.to_datetime(data["Date"])

In [4]:
data = data.drop(columns=['HdrCUSIP', 'PERMCO'])

In [5]:
data.isna().sum()

ticker          0
PERMNO          0
Date            0
DlyVol        738
DlyClose    84650
DlyLow      84650
DlyHigh     84650
DlyOpen     84655
dtype: int64

In [6]:
data = data.dropna().copy()

In [7]:
#Only keep stocks for which trading information is available on all 2768 trading days in the analyzed timeframe
counts = data['PERMNO'].value_counts()

valid_permnos = counts[counts == 2768].index

# Filter the dataframe
data = data[data['PERMNO'].isin(valid_permnos)]

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2726480 entries, 0 to 6663735
Data columns (total 8 columns):
 #   Column    Dtype         
---  ------    -----         
 0   ticker    object        
 1   PERMNO    int64         
 2   Date      datetime64[ns]
 3   DlyVol    float64       
 4   DlyClose  float64       
 5   DlyLow    float64       
 6   DlyHigh   float64       
 7   DlyOpen   float64       
dtypes: datetime64[ns](1), float64(5), int64(1), object(1)
memory usage: 187.2+ MB


In [9]:
data = data.set_index(["Date", "PERMNO"])
data.index.value_counts()

Date        PERMNO
2018-04-30  90470     2
2013-01-02  10026     1
2020-05-01  67467     1
            65008     1
            65285     1
                     ..
2016-08-30  88240     1
            88280     1
            88281     1
            88284     1
2023-12-29  93429     1
Name: count, Length: 2726479, dtype: int64

In [10]:
data = data.drop_duplicates(keep="first")

In [11]:
data = data.reset_index()

In [12]:
data

,Date,PERMNO,ticker,DlyVol,DlyClose,DlyLow,DlyHigh,DlyOpen
0,2013-01-02,10026,JJSF,92378.0,64.3200,63.610,64.8400,64.76
1,2013-01-03,10026,JJSF,51264.0,64.6800,63.360,64.7750,64.21
2,2013-01-04,10026,JJSF,55188.0,64.7700,64.520,65.7700,64.96
3,2013-01-07,10026,JJSF,34634.0,63.4007,63.390,64.7600,64.31
4,2013-01-08,10026,JJSF,83955.0,63.4300,63.000,64.1800,63.65
...,...,...,...,...,...,...,...,...
2726474,2023-12-22,93429,CBOE,420217.0,175.5300,174.785,176.3700,175.85
2726475,2023-12-26,93429,CBOE,433297.0,173.6900,173.500,175.8600,175.86
2726476,2023-12-27,93429,CBOE,475587.0,175.6100,173.640,175.6300,174.11
2726477,2023-12-28,93429,CBOE,682563.0,177.8400,176.040,178.6386,176.04


In [13]:
with open('.././Data/stocks.pkl', 'wb') as file:
    pkl.dump(data, file)